In [29]:
import pandas as pd

In [30]:
melb_data = pd.read_csv('melb_data.csv', sep=',')

In [31]:
df = pd.DataFrame(melb_data)

In [32]:
melb_df = df.copy()

In [81]:
melb_df.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,...,-37.7996,144.9984,Northern Metropolitan,4019.0,"-37.7996, 144.9984",12,46.0,5,St,1
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,...,-37.8079,144.9934,Northern Metropolitan,4019.0,"-37.8079, 144.9934",2,116.0,3,St,0
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,...,-37.8093,144.9944,Northern Metropolitan,4019.0,"-37.8093, 144.9944",3,117.0,5,St,1
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,...,-37.7969,144.9969,Northern Metropolitan,4019.0,"-37.7969, 144.9969",3,47.0,5,other,1
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,...,-37.8072,144.9941,Northern Metropolitan,4019.0,"-37.8072, 144.9941",6,2.0,5,St,1


In [34]:
melb_df.shape

(13580, 23)

In [80]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  object        
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  object        
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  object        
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  float64       
 12  Bathroom       13580 non-null  float64       
 13  Car            13580 non-null  float64       
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

Насколько медианный балл по письму у студентов в расовой группе А отличается от среднего балла по письму у студентов в расовой группе C?

In [36]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

In [37]:
melb_df['MonthSale'] = melb_df['Date'].dt.month

In [38]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']

In [39]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [40]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.day_of_week

In [41]:
melb_df['WeekdaySale'].value_counts()[5] + melb_df['WeekdaySale'].value_counts()[6]

12822

In [42]:
print(melb_df['Address'].nunique())

13378


In [70]:
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

In [71]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [72]:
print(street_types.nunique())

56


In [58]:
replace_dict = {'Avenue': 'Av', 'Boulevard': 'Bvd', 'Parade': 'Pde'}
melb_df['StreetType'] = street_types.apply(lambda k: replace_dict[k] if k in replace_dict.keys() else k)


In [73]:
popular_stypes = street_types.value_counts().nlargest(10).index
popular_stypes

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [74]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')

In [76]:
melb_df['StreetType'].value_counts()

St       8012
Rd       2825
Ct        612
Dr        447
other     420
Av        321
Gr        311
Pde       211
Pl        169
Cr        152
Cl        100
Name: StreetType, dtype: int64

In [183]:
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0

In [79]:
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

In [88]:
av_pr_w_end = melb_df[melb_df['Weekend'] == 1]['Price'].mean()

In [90]:
round(av_pr_w_end)

1081199

In [93]:
popular_agents = melb_df['SellerG'].value_counts().nlargest(49).index
popular_agents

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object')

In [100]:
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_agents else 'other')
price_N_other = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min()
round(price_N_other, 3)

1.298

In [101]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  object        
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  object        
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  object        
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  float64       
 12  Bathroom       13580 non-null  float64       
 13  Car            13580 non-null  float64       
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

In [103]:
popular_suburbs = melb_df['Suburb'].value_counts().nlargest(119).index
popular_suburbs

Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', length=119)

In [106]:
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburbs else 'other')
melb_df['Suburb'].value_counts()

other              2181
Reservoir           359
Richmond            260
Bentleigh East      249
Preston             239
                   ... 
Caulfield North      35
Chadstone            35
Hughesdale           35
Mont Albert          34
Alphington           34
Name: Suburb, Length: 120, dtype: int64

In [107]:
melb_df['Suburb'].astype('category')

0          Abbotsford
1          Abbotsford
2          Abbotsford
3          Abbotsford
4          Abbotsford
             ...     
13575           other
13576    Williamstown
13577    Williamstown
13578    Williamstown
13579      Yarraville
Name: Suburb, Length: 13580, dtype: category
Categories (120, object): ['Abbotsford', 'Aberfeldie', 'Airport West', 'Albert Park', ..., 'Williamstown', 'Windsor', 'Yarraville', 'other']

In [108]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  object        
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  object        
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  object        
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  float64       
 12  Bathroom       13580 non-null  float64       
 13  Car            13580 non-null  float64       
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

In [171]:
bike_data = pd.read_csv('citibike-tripdata.csv')

In [172]:
orig_bike_df = pd.DataFrame(bike_data)

In [173]:
bike_df = orig_bike_df.copy()

In [202]:
bike_df.info()
bike_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype          
---  ------                   --------------   -----          
 0   starttime                300000 non-null  datetime64[ns] 
 1   stoptime                 300000 non-null  datetime64[ns] 
 2   start station id         299831 non-null  float64        
 3   start station name       299831 non-null  object         
 4   start station latitude   300000 non-null  float64        
 5   start station longitude  300000 non-null  float64        
 6   end station id           299831 non-null  float64        
 7   end station name         299831 non-null  object         
 8   end station latitude     300000 non-null  float64        
 9   end station longitude    300000 non-null  float64        
 10  bikeid                   300000 non-null  int64          
 11  usertype                 300000 non-null  object         
 12  ge

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,gender,trip duration,age,Weekday_num,Weekend,time_of_day
0,2018-09-01 00:00:05.269,2018-09-01 00:27:20.634,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1,0 days 00:27:15.365000,38,5,1,night
1,2018-09-01 00:00:11.281,2018-09-01 00:02:23.481,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,0,0 days 00:02:12.200000,49,5,1,night
2,2018-09-01 00:00:20.649,2018-09-01 00:55:58.547,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1,0 days 00:55:37.898000,43,5,1,night
3,2018-09-01 00:00:21.746,2018-09-01 00:07:38.583,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,2,0 days 00:07:16.837000,34,5,1,night
4,2018-09-01 00:00:27.315,2018-09-01 02:21:25.308,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1,0 days 02:20:57.993000,24,5,1,night


In [178]:
bike_df['age'] = 2018 - bike_df['birth year']
bike_df.drop(['birth year'], axis=1, inplace=True)

In [175]:
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])

In [203]:
bike_df['trip duration'] = (bike_df['stoptime']) - (bike_df['starttime'])
bike_df.loc[3, 'trip duration']

Timedelta('0 days 00:07:16.837000')

In [180]:
bike_df['trip duration'].value_counts()

 6      18350
 5      17952
 7      17658
 8      16220
 4      15865
        ...  
-354        1
 509        1
-286        1
 357        1
-650        1
Name: trip duration, Length: 702, dtype: int64

In [204]:
weekday_num = bike_df['starttime'].dt.day_of_week

In [205]:
bike_df['Weekend'] = weekday_num.apply(lambda x: 1 if x == 5 or x == 6 else 0)

In [207]:
bike_df['Weekend'].sum()

115135

In [193]:
def time_of_day(starttime):
    if 0 <= starttime <= 6:
        return 'night'
    elif 6 < starttime <= 12:
        return 'morning'
    elif 12 < starttime <= 18:
        return 'day'
    else:
        return 'evening'

In [194]:
bike_df['time_of_day'] = bike_df['starttime'].dt.hour
bike_df['time_of_day'] = bike_df['time_of_day'].apply(time_of_day)

In [208]:
coef = bike_df[bike_df['time_of_day'] == 'day'].shape[0] / bike_df[bike_df['time_of_day'] == 'night'].shape[0]
round(coef)

9